In [2]:
# Importing Required Libraries 

import cv2
import argparse
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Code For Selecting the Cordinates of the template to be used
# We have to Select Two points in the image that will capture the rectangle part of the image made out from those selected points 

ref_point = []
cropping = False

def shape_selection(event, x, y, flags, param):  
  global ref_point, cropping

  if event == cv2.EVENT_LBUTTONDOWN:
    ref_point = [(x, y)]
    cropping = True
  
  elif event == cv2.EVENT_LBUTTONUP:
    
    ref_point.append((x, y))
    cropping = False
    cv2.rectangle(image, ref_point[0], ref_point[1], (0, 255, 0), 2)
    cv2.imshow("image", image)

image = cv2.imread('indiatoday.png')
clone = image.copy()
cv2.namedWindow("image")
cv2.setMouseCallback("image", shape_selection)

while True:
    cv2.imshow("image", image)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("r"):
        image = clone.copy()    
    elif key == ord("c"):
        break
        
if len(ref_point) == 2:
    crop_img = clone[ref_point[0][1]:ref_point[1][1], ref_point[0][0]:ref_point[1][0]]
    cv2.waitKey(10000)
    print(ref_point)

cv2.destroyAllWindows()

In [11]:
# TV ADBreak Detection Function

def tvAdBreakDetection(templateImage,rtmpURL,coordTemplate):
    sec = 0
    count = 1
    frameRate = 0.3 #Defines After how many seconds a picture must be captured
    x_offset=y_offset=150 # cordinates for watermark
    capture = cv2.VideoCapture(rtmpURL)
    capture.set(cv2.CAP_PROP_POS_MSEC,sec*1000) #Setting For Capturing Image
    success,frame = capture.read()
    template = cv2.imread(templateImage, cv2.IMREAD_GRAYSCALE)
    template = template[coordTemplate[0][1]:coordTemplate[1][1], coordTemplate[0][0]:coordTemplate[1][0]]
    w, h = template.shape[::-1]
    threshold = 0.8
    while success:
        sec = sec + frameRate
        sec = round(sec, 2)
        capture.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
        success,frame = capture.read()
        if success is False:
            capture.release()
            capture = cv2.VideoCapture(rtmpURL)
            capture.set(cv2.CAP_PROP_FPS,1) #Setting For Capturing Image
            success,frame = capture.read()   
        gray_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        matchResult = cv2.matchTemplate(gray_frame,template,cv2.TM_CCOEFF_NORMED)
        frame[y_offset:y_offset+noAdFinalImage.shape[0], x_offset:x_offset+noAdFinalImage.shape[1]] = noAdFinalImage
        loc = np.where(matchResult >= threshold)
        for points in zip(*loc[::-1]):
            cv2.rectangle(frame,points,(points[0] + w, points[1] + h), (0, 255, 0), 3) #Rectangle drawn over part of template matched
            frame[y_offset:y_offset+adFinalImage.shape[0], x_offset:x_offset+adFinalImage.shape[1]] = adFinalImage
        cv2.imshow("Frame", frame)
        cv2.imwrite("Frame%d.jpg" % count, frame)
        count = count + 1
        key = cv2.waitKey(1000)
        if key == ord("x"):
            break
    capture.release()
    cv2.destroyAllWindows()    

In [12]:
# Calling of TV ADBreak Detection Function

rtmpURL = 'rtmp://vidpush-ind.vidgyor.com/it-src/srclive_pc' #Provide URL For Live Stream
templateImage = 'indiatoday.png' # Template Image Source
coordTemplate = [(65, 31), (289, 58)] # Calculated From Above Code
adFinalImage = cv2.imread("ad-final.JPG") # WaterMark For Ad
noAdFinalImage = cv2.imread("no-adfinal.JPG") # Watermark For Live
tvAdBreakDetection(templateImage,rtmpURL,coordTemplate) # Function call